# Create Aurora Borealis Art using GANs and PyTorch

Based on https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/README.rst and https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

In [15]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

arn:aws:iam::019326146125:role/service-role/AmazonSageMaker-ExecutionRole-20190407T165524


In [18]:
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch(entry_point='../train-sagemaker.py',
                            role=role,
                            train_instance_type='ml.p2.xlarge',
                            train_instance_count=1,
                            framework_version='1.0.0',
                            output_path='s3://markstrefford-art-1/aurora-model-1',
                            hyperparameters = {'epochs': 50 })

* Now set up training a model

In [19]:
pytorch_estimator.fit({'training': 's3://markstrefford-art-1/aurora-art-1'})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-04-10-20-48-23-821


2019-04-10 20:48:24 Starting - Starting the training job...
2019-04-10 20:48:25 Starting - Launching requested ML instances......
2019-04-10 20:49:28 Starting - Preparing the instances for training.........
2019-04-10 20:51:16 Downloading - Downloading input data...
2019-04-10 20:51:29 Training - Downloading the training image...
2019-04-10 20:52:13 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-04-10 20:52:15,001 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-04-10 20:52:15,029 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-04-10 20:52:15,634 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-04-10 20:52:15,875 sagemaker-containers INFO     Module train-sagemaker does not provide a setup.py. 
Generating setup.py
2019-04-10 20:52:1

[4/50][0/4]#011Loss_D: 0.0720#011Loss_G: 9.0210#011D(x): 0.9427#011D(G(z)): 0.0060 / 0.0002
[5/50][0/4]#011Loss_D: 0.1463#011Loss_G: 16.3454#011D(x): 0.8917#011D(G(z)): 0.0000 / 0.0000
[6/50][0/4]#011Loss_D: 0.0014#011Loss_G: 8.8170#011D(x): 0.9988#011D(G(z)): 0.0001 / 0.0003
[7/50][0/4]#011Loss_D: 0.0213#011Loss_G: 8.8217#011D(x): 0.9814#011D(G(z)): 0.0006 / 0.0002
[8/50][0/4]#011Loss_D: 0.0078#011Loss_G: 10.5726#011D(x): 0.9925#011D(G(z)): 0.0001 / 0.0000
[9/50][0/4]#011Loss_D: 0.8030#011Loss_G: 20.3478#011D(x): 0.9734#011D(G(z)): 0.4963 / 0.0000
[10/50][0/4]#011Loss_D: 2.5559#011Loss_G: 25.6901#011D(x): 0.9963#011D(G(z)): 0.8843 / 0.0000
[11/50][0/4]#011Loss_D: 0.0052#011Loss_G: 27.0071#011D(x): 0.9949#011D(G(z)): 0.0000 / 0.0000
[12/50][0/4]#011Loss_D: 1.4850#011Loss_G: 25.0927#011D(x): 0.9984#011D(G(z)): 0.7301 / 0.0000
[13/50][0/4]#011Loss_D: 0.0090#011Loss_G: 26.4901#011D(x): 0.9914#011D(G(z)): 0.0000 / 0.0000
[14/50][0/4]#011Loss_D: 0.7133#011Loss_G: 19.7126#011D(x): 0.9969#011

# From examples... 

https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/pytorch_lstm_word_language_model/pytorch_rnn.ipynb

In [10]:
%%bash
wget http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
unzip -n wikitext-2-raw-v1.zip
cd wikitext-2-raw
mv wiki.test.raw test && mv wiki.train.raw train && mv wiki.valid.raw valid

Archive:  wikitext-2-raw-v1.zip
   creating: wikitext-2-raw/
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  


--2019-04-10 19:38:34--  http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
Resolving research.metamind.io.s3.amazonaws.com (research.metamind.io.s3.amazonaws.com)... 52.216.138.171
Connecting to research.metamind.io.s3.amazonaws.com (research.metamind.io.s3.amazonaws.com)|52.216.138.171|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

     0K .......... .......... .......... .......... ..........  1%  353K 13s
    50K .......... .......... .......... .......... ..........  2% 1.39M 8s
   100K .......... .......... .......... .......... ..........  3% 1.35M 6s
   150K .......... .......... .......... .......... ..........  4% 81.1M 5s
   200K .......... .......... .......... .......... ..........  5%  712K 5s
   250K .......... .......... .......... .......... ..........  6% 60.0M 4s
   300K .......... .......... .......... .......... ..........  7% 75.8M 3s
   350K ...

In [11]:
!head -5 wikitext-2-raw/train

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series 

In [12]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-rnn-lstm'

role = sagemaker.get_execution_role()

inputs = sagemaker_session.upload_data(path='wikitext-2-raw', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-eu-west-1-019326146125/sagemaker/DEMO-pytorch-rnn-lstm


In [13]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.0.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    source_dir='../source',
                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 6,
                        'tied': True
                    })

In [14]:
estimator.fit({'training': inputs})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-04-10-19-38-41-369


2019-04-10 19:38:42 Starting - Starting the training job...
2019-04-10 19:38:44 Starting - Launching requested ML instances......
2019-04-10 19:39:47 Starting - Preparing the instances for training.........
2019-04-10 19:41:38 Downloading - Downloading input data
2019-04-10 19:41:38 Training - Downloading the training image.....
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-04-10 19:42:18,029 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-04-10 19:42:18,056 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-04-10 19:42:18,262 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-04-10 19:42:18,579 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-04-10 19:42:18,580 sagemaker-containers INFO     Generating setup.cfg
2019-04-10 19:42:18,580 sagemaker-containe

| epoch   2 |   200/ 2983 batches | lr 20.00 | ms/batch 92.21 | loss  5.83 | ppl   341.13
| epoch   2 |   400/ 2983 batches | lr 20.00 | ms/batch 92.03 | loss  5.81 | ppl   332.28
| epoch   2 |   600/ 2983 batches | lr 20.00 | ms/batch 92.09 | loss  5.60 | ppl   270.83
| epoch   2 |   800/ 2983 batches | lr 20.00 | ms/batch 92.05 | loss  5.62 | ppl   277.03
| epoch   2 |  1000/ 2983 batches | lr 20.00 | ms/batch 92.15 | loss  5.52 | ppl   250.46
| epoch   2 |  1200/ 2983 batches | lr 20.00 | ms/batch 92.04 | loss  5.55 | ppl   256.81
| epoch   2 |  1400/ 2983 batches | lr 20.00 | ms/batch 92.02 | loss  5.60 | ppl   269.26
| epoch   2 |  1600/ 2983 batches | lr 20.00 | ms/batch 92.06 | loss  5.65 | ppl   285.00
| epoch   2 |  1800/ 2983 batches | lr 20.00 | ms/batch 92.16 | loss  5.48 | ppl   240.43
| epoch   2 |  2000/ 2983 batches | lr 20.00 | ms/batch 92.17 | loss  5.49 | ppl   242.47
| epoch   2 |  2200/ 2983 batches | lr 20.00 | ms/batch 92.10 | loss  5.39 | ppl   219.57
| epoch   

| epoch   6 |  2400/ 2983 batches | lr 20.00 | ms/batch 92.76 | loss  4.89 | ppl   133.57
| epoch   6 |  2600/ 2983 batches | lr 20.00 | ms/batch 92.84 | loss  4.92 | ppl   137.33
| epoch   6 |  2800/ 2983 batches | lr 20.00 | ms/batch 92.77 | loss  4.87 | ppl   129.87
-----------------------------------------------------------------------------------------
| end of epoch   6 | time: 286.53s | valid loss  5.55 | valid ppl   258.43
-----------------------------------------------------------------------------------------
Saving the best model: {'epoch': 6, 'lr': 20, 'val_loss': 5.554613220146136, 'val_ppl': 258.4269908341668}

2019-04-10 20:11:28 Uploading - Uploading generated training model=========================================================================================
| End of training | test loss  5.57 | test ppl   261.20
2019-04-10 20:11:25,646 sagemaker-containers INFO     Reporting training SUCCESS

2019-04-10 20:11:49 Completed - Training job completed
Billable seconds: 